## Recursive Prediction

Read multivariable data from a file.<br/>
Perform recursive predictions on the multivariable data.<br/>
Recursive prediction uses the entire library, and prediction at the end of the library to predict Tp=1 step ahead outside the library end.  This prediction vector is then added to the library, prediction indices incremented by one, and a new prediction made.  The process is repeated N_step times. 

In [ ]:
import Jupyter as jp # Do not call jp.init() to create GUI widgets

### Get Data from file

In [ ]:
args            = jp.ParseCmdLine() # Get default arguments
args.path       = '../data/'
args.inputFile  = 'LorenzData1000.csv'
D_Data          = jp.ReadCSV( args )  # Dictionary { data, header }

### Plot Function

In [ ]:
def PlotData( args, data, colNames, xLine ) :

    fig, ax = jp.plt.subplots( 1, 1, figsize = args.figureSize, dpi = 150 )
    colors  = [ 'black', 'blue', 'red', 'green', 'brown', 'magenta' ]
    
    for i in range( 1, len( colNames ) ) :
        ax.plot( data[:,0], data[:,i], label = colNames[i],
                 color = colors[i], linewidth = 2 )
    
    ax.axvline( x = xLine, linewidth = 1, linestyle = 'dashed', color = 'black' )
    ax.legend( loc = 'lower left')
    ax.set( xlabel = args.plotXLabel, ylabel = args.plotYLabel,
            title  = args.inputFile +\
                     '  E=' + str( args.E  ) + ' Tp=' + str( args.Tp ) )
    jp.plt.show()

### Recursive prediction using multivariable data

In [ ]:
import numpy as np

N_step   = 20
data     = D_Data['data'][1:100]    # Subset the data to 100 points
colNames = D_Data['header']
N_Col    = data.shape[1]

# Setup parameters for prediction
args.E          = 5
args.k_NN       = 4
args.Tp         = 1
args.library    = [1,  100]
args.prediction = [90, 100] # Needs to be at the end of data
args.embedded   = True

if args.method == 'smap' :
    args.theta  = 4
    args.k_NN   = 95

jp.AdjustArgs( args )

#---------------------------------------------------------------
# Loop for N_step's
#---------------------------------------------------------------
for i_step in range( 1, N_step ) :
    
    # Create vector to append to data at Tp
    newData = np.zeros( (1, N_Col) ) # 2-D for np.append to data
    
    embedding = data

    # Prediction for each column/variable in the the data as target
    for i_target in range( 1, N_Col ) :
        target = data[ :, i_target ] # Set target
    
        # Note that colNames are not used in Prediction()
        D_Predict = jp.Predict( args, data, None, target, False )

        # Prediction in last row, 3rd col
        newData[ 0, i_target ] = D_Predict['prediction'][ -1, 2 ] 
  
        #print( "{} rho={} mae={}".format( colNames[ i_target ],
        #                        round( rho, 2 ), round( mae, 4 ) ) )

    newData[0,0] = D_Predict['prediction'][ -1, 0 ] # Time value
    print( newData[0,:] )

    # Append this prediction
    data = np.append( data, newData, axis = 0 )

    # Increment for next timestep
    args.library[1]    = args.library[1] + 1
    args.prediction[0] = args.prediction[0] + 1 # Increment both
    args.prediction[1] = args.prediction[1] + 1 # Increment both
    if args.method == 'smap' :
        args.k_NN = args.k_NN + 1
        
    if i_step == 1 :
        xLine = newData[0,0]

outFile = 'out.csv'
np.savetxt( outFile, data, fmt = '%.4f',
            delimiter = ',', header = ','.join( colNames ),
            comments = '' )
        
PlotData( args, data, colNames, xLine )
